In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import mne
import matplotlib.pyplot as plt
import pyvista
import ipywidgets
import ipyevents
import pyvistaqt
import yasa

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_curve, auc
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

import scipy.signal as signal
from scipy.signal import hilbert
from scipy.signal import stft

In [2]:
%matplotlib qt

### CNN Model

In [3]:
def build_cnn_model(input_shape=(500,1)):

    # linear embedding layer
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Three convolutional blocks (like having three pattern detectors)

    # First convolution block, kernel size of 5
    padded1 = tf.keras.layers.ZeroPadding1D(padding=2)(input_layer)
    conv1 = tf.keras.layers.Conv1D(filters=10, kernel_size=5, strides=1, padding='valid')(padded1)
    # each filter learns a different type of short-time feature
    # stride of 1, moves one step at a time
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv1)
    conv1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # K = 2
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

    # Second convolution block, kernel size of 11
    padded2 = tf.keras.layers.ZeroPadding1D(padding=5)(input_layer)
    conv2 = tf.keras.layers.Conv1D(filters=10, kernel_size=11, strides=1, padding='valid')(padded2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv2)
    conv2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)

    # Third convolution block, kernel size of 21
    padded3 = tf.keras.layers.ZeroPadding1D(padding=10)(input_layer)
    conv3 = tf.keras.layers.Conv1D(filters=10, kernel_size=21, strides=1, padding='valid')(padded3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv3)
    conv3 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)

    # Concatenate the outputs of all blocks
    concatenated = tf.keras.layers.Concatenate()([conv1, conv2, conv3])

    # GRU Layer
    gru = tf.keras.layers.GRU(64)(concatenated)

    # Fully connected (dense) layer
    dense = tf.keras.layers.Dense(64, activation='relu')(gru)
    # add a Dropout layer to prevent overfitting
    #dense = tf.keras.layers.Dropout(0.5)(dense)

    # Two softmax outputs for dual-task classification
    #output_task1 = tf.keras.layers.Dense(2, activation='softmax', name='task1')(dense)
    #output_task2 = tf.keras.layers.Dense(2, activation='softmax', name='task2')(dense)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    # Create the model
    #model = tf.keras.models.Model(inputs=input_layer, outputs=[output_task1, output_task2])
    model = tf.keras.models.Model(inputs=input_layer, outputs=output)

    # Compile the model
    #model.compile(optimizer='adam', loss={'task1': 'categorical_crossentropy', 'task2': 'categorical_crossentropy'}, metrics={'task1': 'accuracy', 'task2': 'accuracy'})
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Return the compiled model
    return model

# show model architecture
input_shape = (500, 1)
model = build_cnn_model(input_shape)
model.summary()

C:\Users\roseb\anaconda3\envs\msc_research_project\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 500, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d      │ (None, 504, 1)    │          0 │ input_layer[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_1    │ (None, 510, 1)    │          0 │ input_layer[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_2    │ (None, 520, 1)    │          0 │ input_layer[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 500, 10)   │         60 │ zero_padding1d[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 500, 10)   │        120 │ zero_padding1d_1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 500, 10)   │        220 │ zero_padding1d_2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 500, 10)   │          0 │ conv1d[0][0]      │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 500, 10)   │          0 │ conv1d_1[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 500, 10)   │          0 │ conv1d_2[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 250, 10)   │          0 │ leaky_re_lu[0][0] │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 250, 10)   │          0 │ leaky_re_lu_1[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 250, 10)   │          0 │ leaky_re_lu_2[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 250, 10)   │         40 │ max_pooling1d[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 10)   │         40 │ max_pooling1d_1[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 10)   │         40 │ max_pooling1d_2[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 250, 30)   │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 23,177 (90.54 KB)

 Trainable params: 23,117 (90.30 KB)

 Non-trainable params: 60 (240.00 B)

In [26]:
def build_cnn_model_downsampled(input_shape=(300,1)):

    # linear embedding layer
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Three convolutional blocks (like having three pattern detectors)

    # First convolution block, kernel size of 5
    padded1 = tf.keras.layers.ZeroPadding1D(padding=2)(input_layer)
    conv1 = tf.keras.layers.Conv1D(filters=10, kernel_size=5, strides=1, padding='valid')(padded1)
    # each filter learns a different type of short-time feature
    # stride of 1, moves one step at a time
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv1)
    conv1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # K = 2
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

    # Second convolution block, kernel size of 11
    padded2 = tf.keras.layers.ZeroPadding1D(padding=5)(input_layer)
    conv2 = tf.keras.layers.Conv1D(filters=10, kernel_size=11, strides=1, padding='valid')(padded2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv2)
    conv2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)

    # Third convolution block, kernel size of 21
    padded3 = tf.keras.layers.ZeroPadding1D(padding=10)(input_layer)
    conv3 = tf.keras.layers.Conv1D(filters=10, kernel_size=21, strides=1, padding='valid')(padded3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv3)
    conv3 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)

    # Concatenate the outputs of all blocks
    concatenated = tf.keras.layers.Concatenate()([conv1, conv2, conv3])

    # GRU Layer
    gru = tf.keras.layers.GRU(64)(concatenated)

    # Fully connected (dense) layer
    dense = tf.keras.layers.Dense(64, activation='relu')(gru)
    # add a Dropout layer to prevent overfitting
    #dense = tf.keras.layers.Dropout(0.5)(dense)

    # Two softmax outputs for dual-task classification
    #output_task1 = tf.keras.layers.Dense(2, activation='softmax', name='task1')(dense)
    #output_task2 = tf.keras.layers.Dense(2, activation='softmax', name='task2')(dense)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    # Create the model
    #model = tf.keras.models.Model(inputs=input_layer, outputs=[output_task1, output_task2])
    model = tf.keras.models.Model(inputs=input_layer, outputs=output)

    # Compile the model
    #model.compile(optimizer='adam', loss={'task1': 'categorical_crossentropy', 'task2': 'categorical_crossentropy'}, metrics={'task1': 'accuracy', 'task2': 'accuracy'})
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Return the compiled model
    return model

In [39]:
def build_2d_cnn_model(input_shape=(65, 25, 1)):
    # Input: STFT spectrogram (freq_bins x time_bins x 1)
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # First convolution block, kernel size (3x3)
    padded1 = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(input_layer)
    conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=(3, 3), strides=(1, 1), padding='valid')(padded1)
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv1)
    conv1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

    # Second convolution block, kernel size (5x5)
    padded2 = tf.keras.layers.ZeroPadding2D(padding=(2, 2))(input_layer)
    conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=(5, 5), strides=(1, 1), padding='valid')(padded2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv2)
    conv2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)

    # Third convolution block, kernel size (7x7)
    padded3 = tf.keras.layers.ZeroPadding2D(padding=(3, 3))(input_layer)
    conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=(7, 7), strides=(1, 1), padding='valid')(padded3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv3)
    conv3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)

    # Concatenate all three conv outputs
    concatenated = tf.keras.layers.Concatenate(axis=-1)([conv1, conv2, conv3])

    # Flatten before passing to RNN
    reshaped = tf.keras.layers.Reshape((-1, concatenated.shape[-1]))(concatenated)

    # GRU layer for temporal modeling over time slices
    gru = tf.keras.layers.GRU(64)(reshaped)

    # Fully connected dense layer
    dense = tf.keras.layers.Dense(64, activation='relu')(gru)

    # Output layer for binary classification
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    # Define and compile model
    model = tf.keras.models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# show model architecture
input_shape = (65, 25, 1)
cnn_2d_model = build_2d_cnn_model(input_shape)
cnn_2d_model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 65, 25, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 67, 27, 1) │          0 │ input_layer_5[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_4    │ (None, 69, 29, 1) │          0 │ input_layer_5[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_5    │ (None, 71, 31, 1) │          0 │ input_layer_5[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 65, 25,    │        100 │ zero_padding2d_3… │
│                     │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 65, 25,    │        260 │ zero_padding2d_4… │
│                     │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 65, 25,    │        500 │ zero_padding2d_5… │
│                     │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_12      │ (None, 65, 25,    │          0 │ conv2d_3[0][0]    │
│ (LeakyReLU)         │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_13      │ (None, 65, 25,    │          0 │ conv2d_4[0][0]    │
│ (LeakyReLU)         │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_14      │ (None, 65, 25,    │          0 │ conv2d_5[0][0]    │
│ (LeakyReLU)         │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 32, 12,    │          0 │ leaky_re_lu_12[0… │
│ (MaxPooling2D)      │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 32, 12,    │          0 │ leaky_re_lu_13[0… │
│ (MaxPooling2D)      │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 32, 12,    │          0 │ leaky_re_lu_14[0… │
│ (MaxPooling2D)      │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 12,    │         40 │ max_pooling2d_3[… │
│ (BatchNormalizatio… │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 12,    │         40 │ max_pooling2d_4[… │
│ (BatchNormalizatio… │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 12,    │         40 │ max_pooling2d_5[… │
│ (BatchNormalizatio… │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 32, 12,    │          0 │ batch_normalizat

 Total params: 23,637 (92.33 KB)

 Trainable params: 23,577 (92.10 KB)

 Non-trainable params: 60 (240.00 B)

### Spindle detection function

In [4]:
def detect_spindles_times(eeg_raw, do_filter=True, do_downsample=False, downsample_rate=100):
    
    # 1. Filter between 12 and 16 Hz
    
    data = eeg_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=12, h_freq=16)
    
    # 2. Downsample at 100 Hz (100 samples per second)

    if do_downsample:
        data.resample(downsample_rate)
    
    sfreq = data.info['sfreq']  
    channel_data = data.get_data()[0]
    # extract the filtered data
    
    
    # 3: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = hilbert(channel_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 4: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 5. Define spindle detection threshold

    threshold = np.percentile(smoothed_envelope, 75)
    spindle_threshold = smoothed_envelope > threshold
    #threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    #spindle_threshold = smoothed_envelope > threshold
    # threshold is 75th percentile of the smoothed envelope
    # will look at the duration later
    
    # 6. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                # so starting from the second index
                # and comparing each index to the one before
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = channel_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    spindles.append((start_idx / sfreq, end_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = channel_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop

        # then need to process the final spindle
        end_idx = above_threshold[-1]
        duration = (end_idx - start_idx) / sfreq
        if 0.5 <= duration <= 3:
            segment = channel_data[start_idx:end_idx]
            peak_idx = start_idx + np.argmax(segment)
            spindles.append((start_idx / sfreq, end_idx / sfreq))

            before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
            after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
            aligned_segment = channel_data[before_peak_idx:after_peak_idx]
            stacked_spindles.append(aligned_segment)
    
    return spindles
    

def detect_spindles_peaks(eeg_raw, do_filter=True, do_downsample=False, downsample_rate=100):
    
    # 1. Filter between 12 and 16 Hz
    
    data = eeg_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=12, h_freq=16)
    
    # 2. Downsample at 100 Hz (100 samples per second)
    
    if do_downsample:
        data.resample(downsample_rate)
        
    sfreq = data.info['sfreq']  
    # update to new sampling frequency
    # because used later in the code
    channel_data = data.get_data()[0]
    # extract the filtered data
    
    # 3: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = hilbert(channel_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 4: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 5. Define spindle detection threshold

    threshold = np.percentile(smoothed_envelope, 75)
    spindle_threshold = smoothed_envelope > threshold
    # 75th percentile as criteria

    #threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    #spindle_threshold = smoothed_envelope > threshold
    
    # 6. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = channel_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    #spindles.append(f"Spindle detected from {start_idx / sfreq:.2f}s to {end_idx / sfreq:.2f}s, peak at {peak_idx / sfreq:.2f}s")
                    spindles.append((peak_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = channel_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop

        # then need to process the final spindle
        end_idx = above_threshold[-1]
        duration = (end_idx - start_idx) / sfreq
        if 0.5 <= duration <= 3:
            segment = channel_data[start_idx:end_idx]
            peak_idx = start_idx + np.argmax(segment)
            spindles.append((peak_idx / sfreq))

            before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
            after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
            aligned_segment = channel_data[before_peak_idx:after_peak_idx]
            stacked_spindles.append(aligned_segment)

    
    return spindles

### Epochs function

In [5]:
def create_fixed_length_epochs(raw, duration=3.0, overlap=0.0, preload=True, reject_by_annotation=False):

    return mne.make_fixed_length_epochs(
        raw,
        duration=duration,
        overlap=overlap,
        preload=preload,
        reject_by_annotation=reject_by_annotation
    )
# function mne.make_fixed_length_epochs takes into account the sampling frequency of the data

def label_spindle_epochs(epochs, spindle_starts, spindle_ends, epoch_length_sec=3.0):

    epoch_starts = np.arange(len(epochs)) * epoch_length_sec
    # new np array with the start time of each epoch
    # epoch_starts[i] is the start time of each epoch

    epoch_labels = np.zeros(len(epochs), dtype=int)
    # initialize all the labels as 0 initially

    for start, end in zip(spindle_starts, spindle_ends):
        # loop through the start and end times of detected spindles by YASA
        for i, epoch_start in enumerate(epoch_starts):
            # loop through the one-second epochs that are not labelled yet
            epoch_end = epoch_start + epoch_length_sec
            # for each epoch, calculate the epoch end time
            # which is epoch_start + length of epoch
            # so now have the time range of each epoch
            if (start < epoch_end) and (end > epoch_start):
                # if the spindle started before the epoch ends
                # and the spindle ended after the epoch started
                epoch_labels[i] = 1
                
    return epoch_labels

### Importing data

In [18]:
# file paths
train_file = r"C:\EEG DATA\combined_sets\train_raw.fif"
test_file = r"C:\EEG DATA\combined_sets\test_raw.fif"

# load raw files
train_raw = mne.io.read_raw_fif(train_file, preload=True)
test_raw = mne.io.read_raw_fif(test_file, preload=True)

Opening raw data file C:\EEG DATA\combined_sets\train_raw.fif...
Isotrak not found
    Range : 1470000 ... 23295072 =   2940.000 ... 46590.144 secs
Ready.
Reading 0 ... 21825072  =      0.000 ... 43650.144 secs...
Opening raw data file C:\EEG DATA\combined_sets\test_raw.fif...
Isotrak not found
    Range : 825000 ... 17985049 =   1650.000 ... 35970.098 secs
Ready.
Reading 0 ... 17160049  =      0.000 ... 34320.098 secs...


## With raw data

### Spindle detection

In [19]:
spindles_train_times_raw_downsampled = detect_spindles_times(train_raw, do_filter=True, do_downsample=True)
spindles_test_times_raw_downsampled = detect_spindles_times(test_raw, do_filter=True, do_downsample=True)

spindles_starts_train_raw_downsampled, spindles_ends_train_raw_downsampled = zip(*spindles_train_times_raw_downsampled) if spindles_train_times_raw_downsampled else([],[])
spindles_starts_test_raw_downsampled, spindles_ends_test_raw_downsampled = zip(*spindles_test_times_raw_downsampled) if spindles_test_times_raw_downsampled else([],[])

print(len(spindles_starts_train_raw_downsampled))
print(len(spindles_ends_train_raw_downsampled))

print(len(spindles_starts_test_raw_downsampled))
print(len(spindles_ends_test_raw_downsampled))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 551 samples (1.102 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 50 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 pass

### Downsample

In [20]:
train_raw_downsampled = train_raw.copy().resample(100)
test_raw_downsampled = test_raw.copy().resample(100)

print(train_raw_downsampled.info['sfreq'])
print(test_raw_downsampled.info['sfreq'])

100.0
100.0


### Epoch the data

In [21]:
epochs_train_raw_downsampled = create_fixed_length_epochs(train_raw_downsampled)
epochs_test_raw_downsampled = create_fixed_length_epochs(test_raw_downsampled)
print(epochs_train_raw_downsampled.get_data().shape)
print(epochs_test_raw_downsampled.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped
(14550, 1, 300)
(11440, 1, 300)


### Labels for 3-second epochs

In [23]:
%%time

# Train set

epoch_labels_train_raw_downsampled = label_spindle_epochs(epochs_train_raw_downsampled, spindles_starts_train_raw_downsampled, spindles_ends_train_raw_downsampled)

print(f"Train data first 10 spindles: {spindles_starts_train_raw_downsampled[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_raw_downsampled[:10]}")

# Test set

epoch_labels_test_raw_downsampled = label_spindle_epochs(epochs_test_raw_downsampled, spindles_starts_test_raw_downsampled, spindles_ends_test_raw_downsampled)

print(f"\nTest data first 10 spindles: {spindles_starts_test_raw_downsampled[:10]}")
print(f"Test labels first 10 epochs: {epoch_labels_test_raw_downsampled[:10]}")

Train data first 10 spindles: (np.float64(0.05), np.float64(4.95), np.float64(10.01), np.float64(19.41), np.float64(27.23), np.float64(30.01), np.float64(31.05), np.float64(41.95), np.float64(47.79), np.float64(53.14))
Train labels first 10 epochs: [1 1 1 1 0 0 1 0 0 1]

Test data first 10 spindles: (np.float64(29.89), np.float64(49.64), np.float64(61.33), np.float64(78.56), np.float64(87.9), np.float64(88.5), np.float64(97.5), np.float64(112.37), np.float64(119.92), np.float64(147.43))
Test labels first 10 epochs: [0 0 0 0 0 0 0 0 0 1]
CPU times: total: 29.8 s
Wall time: 29.9 s


### Prepare EEG data for CNN input

#### X and y train and test sets

In [24]:
# Reshape arrays

epochs_train_np_raw_downsampled = np.array(epochs_train_raw_downsampled).reshape(len(epochs_train_raw_downsampled), -1, 1)
# number of epochs N, sampling frequency (time dimension automatically inferred), channel dimension
epochs_test_np_raw_downsampled = np.array(epochs_test_raw_downsampled).reshape(len(epochs_test_raw_downsampled), -1, 1)
                                                                 
# Define X and y sets
                                                                 
X_train_raw_downsampled = epochs_train_np_raw_downsampled
y_train_raw_downsampled = epoch_labels_train_raw_downsampled

X_test_raw_downsampled = epochs_test_np_raw_downsampled
y_test_raw_downsampled = epoch_labels_test_raw_downsampled

# Print shapes

print(f"X_train shape: {X_train_raw_downsampled.shape}")
print(f"y_train shape: {y_train_raw_downsampled.shape}")

print(f"\nX_test shape: {X_test_raw_downsampled.shape}")
print(f"y_test shape: {y_test_raw_downsampled.shape}")

X_train shape: (14550, 300, 1)
y_train shape: (14550,)

X_test shape: (11440, 300, 1)
y_test shape: (11440,)


### Train the model

In [25]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

In [27]:
# show model architecture
input_shape = (300, 1)
model_raw_downsampled = build_cnn_model_downsampled(input_shape)
model_raw_downsampled.summary()

C:\Users\roseb\anaconda3\envs\msc_research_project\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 300, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_3    │ (None, 304, 1)    │          0 │ input_layer_1[0]… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_4    │ (None, 310, 1)    │          0 │ input_layer_1[0]… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_5    │ (None, 320, 1)    │          0 │ input_layer_1[0]… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 300, 10)   │         60 │ zero_padding1d_3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 300, 10)   │        120 │ zero_padding1d_4… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 300, 10)   │        220 │ zero_padding1d_5… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 300, 10)   │          0 │ conv1d_3[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_4       │ (None, 300, 10)   │          0 │ conv1d_4[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_5       │ (None, 300, 10)   │          0 │ conv1d_5[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 150, 10)   │          0 │ leaky_re_lu_3[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 150, 10)   │          0 │ leaky_re_lu_4[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_5     │ (None, 150, 10)   │          0 │ leaky_re_lu_5[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 10)   │         40 │ max_pooling1d_3[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 10)   │         40 │ max_pooling1d_4[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 10)   │         40 │ max_pooling1d_5[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 150, 30)   │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 23,177 (90.54 KB)

 Trainable params: 23,117 (90.30 KB)

 Non-trainable params: 60 (240.00 B)

In [28]:
%%time

training_info_raw_downsampled = model_raw_downsampled.fit(X_train_raw_downsampled, y_train_raw_downsampled, validation_split=0.2, epochs=30, batch_size=128, callbacks=[early_stop])

Epoch 1/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 138ms/step - accuracy: 0.5069 - loss: 0.6993 - val_accuracy: 0.5100 - val_loss: 0.6936
Epoch 2/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 133ms/step - accuracy: 0.5617 - loss: 0.6774 - val_accuracy: 0.5220 - val_loss: 0.6767
Epoch 3/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 133ms/step - accuracy: 0.6474 - loss: 0.6403 - val_accuracy: 0.7079 - val_loss: 0.5571
Epoch 4/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 128ms/step - accuracy: 0.6501 - loss: 0.6229 - val_accuracy: 0.6945 - val_loss: 0.5874
Epoch 5/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 132ms/step - accuracy: 0.7084 - loss: 0.5673 - val_accuracy: 0.7784 - val_loss: 0.4711
Epoch 6/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 124ms/step - accuracy: 0.7292 - loss: 0.5381 - val_accuracy: 0.7117 - val_loss: 0.5794
Epoch 7/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 117ms/step - accuracy: 0.7161 - loss: 0.5561 - val_accuracy: 0.7849 - val_loss: 0.4762
Epoch 8/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.7858 - loss: 0.4692 - val_accu

In [30]:
def plot_training_history(training_info):
  fig, axs = plt.subplots(1, 2, figsize=(16, 5))
  axs[0].plot(training_info.history['loss'], label="training set")
  axs[0].plot(training_info.history['val_loss'], label="validation set")
  axs[0].set_xlabel("Epoch")
  axs[0].set_ylabel("Loss")
  axs[0].grid(True)
  axs[0].legend()
  try:
    axs[1].plot(training_info.history['accuracy'], label="training set")
    axs[1].plot(training_info.history['val_accuracy'], label="validation set")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].grid(True)
    axs[1].legend()
  except:
    pass
  plt.show()

plot_training_history(training_info_raw_downsampled)

In [31]:
model_raw_downsampled.evaluate(X_test_raw_downsampled, y_test_raw_downsampled)

358/358 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7779 - loss: 0.4784


[0.4881678521633148, 0.7668706178665161]

In [33]:
y_pred = model_raw_downsampled.predict(X_test_raw_downsampled)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_raw_downsampled, y_pred_labels))
print(classification_report(y_test_raw_downsampled, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
[[4173  981]
 [1686 4600]]
              precision    recall  f1-score   support

           0       0.71      0.81      0.76      5154
           1       0.82      0.73      0.78      6286

    accuracy                           0.77     11440
   macro avg       0.77      0.77      0.77     11440
weighted avg       0.77      0.77      0.77     11440



In [34]:
# Flatten in case y_pred has shape (n_samples, 1)
y_pred_proba = y_pred.ravel()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_raw_downsampled, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## With filtered data

In [7]:
# Apply bandpass filter between 12 and 16 Hz
# to compare performance of filtered dataset to unfiltered one
train_filtered = train_raw.copy().filter(l_freq=12, h_freq=16)
test_filtered = test_raw.copy().filter(l_freq=12, h_freq=16)

# Downsample to 100 Hz
train_filtered.resample(100)
test_filtered.resample(100)

Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 551 samples (1.102 s)

Filtering raw data in 50 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passban

<Raw | test_raw.fif, 1 x 3432010 (34320.1 s), ~26.2 MiB, data loaded>

### Spindle detection

In [8]:
spindles_train_times_filtered = detect_spindles_times(train_filtered, do_filter=False, do_downsample=False)
spindles_test_times_filtered = detect_spindles_times(test_filtered, do_filter=False, do_downsample=False)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [9]:
print(len(spindles_train_times_filtered))
print(len(spindles_test_times_filtered))

7123
5794


In [10]:
spindles_starts_train_filtered, spindles_ends_train_filtered = zip(*spindles_train_times_filtered) if spindles_train_times_filtered else([],[])
spindles_starts_test_filtered, spindles_ends_test_filtered = zip(*spindles_test_times_filtered) if spindles_test_times_filtered else([],[])

In [11]:
print(len(spindles_starts_train_filtered))
print(len(spindles_ends_train_filtered))

print(len(spindles_starts_test_filtered))
print(len(spindles_ends_test_filtered))

7123
7123
5794
5794


### Epoch the data

In [12]:
epochs_train_filtered = create_fixed_length_epochs(train_filtered)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped


In [13]:
epochs_test_filtered = create_fixed_length_epochs(test_filtered)

Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped


In [14]:
print(epochs_train_filtered.get_data().shape)
print(epochs_test_filtered.get_data().shape)

(14550, 1, 300)
(11440, 1, 300)


### Labels for 3-second epochs

In [15]:
%%time

# Train set

epoch_labels_train_filtered = label_spindle_epochs(epochs_train_filtered, spindles_starts_train_filtered, spindles_ends_train_filtered)

print(f"Train data first 10 spindles: {spindles_starts_train_filtered[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_filtered[:10]}")

# Test set

epoch_labels_test_filtered = label_spindle_epochs(epochs_test_filtered, spindles_starts_test_filtered, spindles_ends_test_filtered)

print(f"\nTest data first 10 spindles: {spindles_starts_test_filtered[:10]}")
print(f"Test labels first 10 epochs: {epoch_labels_test_filtered[:10]}")

Train data first 10 spindles: (np.float64(0.05), np.float64(4.95), np.float64(10.01), np.float64(19.41), np.float64(27.23), np.float64(30.01), np.float64(31.05), np.float64(41.95), np.float64(47.79), np.float64(53.14))
Train labels first 10 epochs: [1 1 1 1 0 0 1 0 0 1]

Test data first 10 spindles: (np.float64(29.89), np.float64(49.64), np.float64(61.33), np.float64(78.56), np.float64(87.9), np.float64(88.5), np.float64(97.5), np.float64(112.37), np.float64(119.92), np.float64(147.43))
Test labels first 10 epochs: [0 0 0 0 0 0 0 0 0 1]
CPU times: total: 53.3 s
Wall time: 54 s


### Filter and downsample

In [ ]:
# Apply bandpass filter between 12 and 16 Hz
# to compare performance of filtered dataset to unfiltered one
train_filtered = train_raw.copy().filter(l_freq=12, h_freq=16)
test_filtered = test_raw.copy().filter(l_freq=12, h_freq=16)

# Downsample to 100 Hz
train_filtered.resample(100)
test_filtered.resample(100)

### Prepare EEG data for CNN input

#### X and y train and test sets

In [16]:
# Reshape arrays

epochs_train_np_filtered = np.array(epochs_train_filtered).reshape(len(epochs_train_filtered), -1, 1)
# number of epochs N, sampling frequency (time dimension automatically inferred), channel dimension
epochs_test_np_filtered = np.array(epochs_test_filtered).reshape(len(epochs_test_filtered), -1, 1)
                                                                 
# Define X and y sets
                                                                 
X_train_filtered = epochs_train_np_filtered
y_train_filtered = epoch_labels_train_filtered

X_test_filtered = epochs_test_np_filtered
y_test_filtered = epoch_labels_test_filtered

# Print shapes

print(f"X_train shape: {X_train_filtered.shape}")
print(f"y_train shape: {y_train_filtered.shape}")

print(f"\nX_test shape: {X_test_filtered.shape}")
print(f"y_test shape: {y_test_filtered.shape}")
                                                                 

X_train shape: (14550, 300, 1)
y_train shape: (14550,)

X_test shape: (11440, 300, 1)
y_test shape: (11440,)


### Train the model

In [ ]:
%%time

training_info = model.fit(X_train_filtered, y_train_filtered, validation_split=0.2, epochs=30, batch_size=128, callbacks=[early_stop])

#### Inspect class imbalance

In [17]:
# Count spindle and non-spindle labels in train and test sets
unique_train_filtered, counts_train_filtered = np.unique(y_train_filtered, return_counts=True)
unique_test_filtered, counts_test_filtered = np.unique(y_test_filtered, return_counts=True)

print("Train label distribution:")
for label, count in zip(unique_train_filtered, counts_train_filtered):
    print(f"Label {label}: {count}")

print("\nTest label distribution:")
for label, count in zip(unique_test_filtered, counts_test_filtered):
    print(f"Label {label}: {count}")

Train label distribution:
Label 0: 7103
Label 1: 7447

Test label distribution:
Label 0: 5154
Label 1: 6286


#### Training

In [19]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

In [20]:
%%time

training_info = model.fit(X_train_filtered, y_train_filtered, validation_split=0.2, epochs=30, batch_size=128, callbacks=[early_stop])

Epoch 1/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 23s 170ms/step - accuracy: 0.5920 - loss: 0.6493 - val_accuracy: 0.5780 - val_loss: 0.7163
Epoch 2/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 15s 163ms/step - accuracy: 0.7226 - loss: 0.5401 - val_accuracy: 0.6182 - val_loss: 0.6570
Epoch 3/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 16s 171ms/step - accuracy: 0.6273 - loss: 0.6229 - val_accuracy: 0.6533 - val_loss: 0.6407
Epoch 4/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 15s 164ms/step - accuracy: 0.6633 - loss: 0.5930 - val_accuracy: 0.8553 - val_loss: 0.3555
Epoch 5/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 15s 166ms/step - accuracy: 0.8434 - loss: 0.3481 - val_accuracy: 0.8842 - val_loss: 0.2843
Epoch 6/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 16s 172ms/step - accuracy: 0.8757 - loss: 0.2922 - val_accuracy: 0.8955 - val_loss: 0.2463
Epoch 7/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 15s 164ms/step - accuracy: 0.8914 - loss: 0.2600 - val_accuracy: 0.8938 - val_loss: 0.2516
Epoch 8/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 15s 167ms/step - accuracy: 0.8959 - loss: 0.2424 - val_accu

In [21]:
def plot_training_history(training_info):
  fig, axs = plt.subplots(1, 2, figsize=(16, 5))
  axs[0].plot(training_info.history['loss'], label="training set")
  axs[0].plot(training_info.history['val_loss'], label="validation set")
  axs[0].set_xlabel("Epoch")
  axs[0].set_ylabel("Loss")
  axs[0].grid(True)
  axs[0].legend()
  try:
    axs[1].plot(training_info.history['accuracy'], label="training set")
    axs[1].plot(training_info.history['val_accuracy'], label="validation set")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].grid(True)
    axs[1].legend()
  except:
    pass
  plt.show()

plot_training_history(training_info)

In [22]:
model.evaluate(X_test_filtered, y_test_filtered)

358/358 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.8999 - loss: 0.2338 


[0.24101856350898743, 0.8963286876678467]

### Metrics

In [23]:
y_pred = model.predict(X_test_filtered)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_filtered, y_pred_labels))
print(classification_report(y_test_filtered, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step
[[5042  112]
 [1074 5212]]
              precision    recall  f1-score   support

           0       0.82      0.98      0.89      5154
           1       0.98      0.83      0.90      6286

    accuracy                           0.90     11440
   macro avg       0.90      0.90      0.90     11440
weighted avg       0.91      0.90      0.90     11440



In [24]:
# Flatten in case y_pred has shape (n_samples, 1)
y_pred_proba = y_pred.ravel()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_filtered, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## With STFT

In [13]:
spindles_train_times_stft = detect_spindles_times(train_raw, do_filter=True, do_downsample=False)
spindles_test_times_stft = detect_spindles_times(test_raw, do_filter=True, do_downsample=False)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 551 samples (1.102 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 50 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 pass

In [14]:
spindles_starts_train_stft, spindles_ends_train_stft = zip(*spindles_train_times_stft) if spindles_train_times_stft else([],[])
spindles_starts_test_stft, spindles_ends_test_stft = zip(*spindles_test_times_stft) if spindles_test_times_stft else([],[])

print(len(spindles_starts_train_stft))
print(len(spindles_ends_train_stft))

print(len(spindles_starts_test_stft))
print(len(spindles_ends_test_stft))

7203
7203
5852
5852


### Epoch the data

In [22]:
epochs_train_stft = create_fixed_length_epochs(train_raw)
epochs_test_stft = create_fixed_length_epochs(test_raw)

print(epochs_train_stft.get_data().shape)
print(epochs_test_stft.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 1500 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 1500 original time points ...
0 bad epochs dropped
(14550, 1, 1500)
(11440, 1, 1500)


### Labels for 3-second epochs

In [23]:
%%time

# Train set

epoch_labels_train_stft = label_spindle_epochs(epochs_train_stft, spindles_starts_train_stft, spindles_ends_train_stft)

print(f"Train data first 10 spindles: {spindles_starts_train_stft[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_stft[:10]}")

# Test set

epoch_labels_test_stft = label_spindle_epochs(epochs_test_stft, spindles_starts_test_stft, spindles_ends_test_stft)

print(f"\nTest data first 10 spindles: {spindles_starts_test_stft[:10]}")
print(f"Test labels first 10 epochs: {epoch_labels_test_stft[:10]}")

Train data first 10 spindles: (np.float64(0.05), np.float64(4.944), np.float64(10.006), np.float64(19.4), np.float64(27.224), np.float64(30.008), np.float64(31.048), np.float64(41.948), np.float64(47.788), np.float64(53.138))
Train labels first 10 epochs: [1 1 1 1 0 0 1 0 0 1]

Test data first 10 spindles: (np.float64(29.878), np.float64(49.632), np.float64(61.326), np.float64(78.552), np.float64(87.896), np.float64(88.492), np.float64(97.486), np.float64(112.366), np.float64(119.91), np.float64(147.422))
Test labels first 10 epochs: [0 0 0 0 0 0 0 0 0 1]
CPU times: total: 27.6 s
Wall time: 27.7 s


### Apply STFT to each epoch

#### get rid of channel dimension

In [30]:
epochs_train_stft = np.squeeze(epochs_train_stft)
epochs_test_stft = np.squeeze(epochs_test_stft)

print(epochs_train_stft.shape)
print(epochs_test_stft.shape)

(14550, 1500)
(11440, 1500)


In [31]:
fs = 500  
nperseg = 128  # ~0.26 sec window
noverlap = 64  # 50% overlap


epochs_train_stft_transformed = []

for epoch in epochs_train_stft:
    f, t, Zxx = stft(epoch, fs=fs, nperseg=nperseg, noverlap=noverlap)
    spectrogram = np.abs(Zxx)  
    epochs_train_stft_transformed.append(spectrogram)

epochs_test_stft_transformed = []

for epoch in epochs_test_stft:
    f, t, Zxx = stft(epoch, fs=fs, nperseg=nperseg, noverlap=noverlap)
    spectrogram = np.abs(Zxx)  # Shape: (freq_bins, time_bins)
    epochs_test_stft_transformed.append(spectrogram)
    
# convert into numpy arrays
epochs_train_stft_transformed = np.array(epochs_train_stft_transformed)
epochs_test_stft_transformed = np.array(epochs_test_stft_transformed)

print("Train STFT shape:", epochs_train_stft_transformed.shape)
print("Test STFT shape:", epochs_test_stft_transformed.shape)


Train STFT shape: (14550, 65, 25)
Test STFT shape: (11440, 65, 25)


### X and y train and test sets

In [32]:
# Define X and y sets and reshape

X_train_stft = epochs_train_stft_transformed[..., np.newaxis]  # Shape: (14550, 65, 25, 1)
y_train_stft = epoch_labels_train_stft

X_test_stft = epochs_test_stft_transformed[..., np.newaxis]    # Shape: (11440, 65, 25, 1)
y_test_stft = epoch_labels_test_stft
                                                                 

# Print shapes

print(f"X_train shape: {X_train_stft.shape}")
print(f"y_train shape: {y_train_stft.shape}")

print(f"\nX_test shape: {X_test_stft.shape}")
print(f"y_test shape: {y_test_stft.shape}")

X_train shape: (14550, 65, 25, 1)
y_train shape: (14550,)

X_test shape: (11440, 65, 25, 1)
y_test shape: (11440,)


### Normalization of data

In [53]:
# this scales the spectrogram in range 0,1

X_train_stft_norm = np.array([
    (epoch - np.min(epoch)) / (np.max(epoch) - np.min(epoch) + 1e-8)
    for epoch in X_train_stft
])

X_test_stft_norm = np.array([
    (epoch - np.min(epoch)) / (np.max(epoch) - np.min(epoch) + 1e-8)
    for epoch in X_test_stft
])

In [66]:
# should have values between 0 and 1 

print("Max train value after normalization:", np.max(X_train_stft_norm))
print("Min train value:", np.min(X_train_stft_norm))

print("Max train value after normalization:", np.max(X_test_stft_norm))
print("Min train value:", np.min(X_test_stft_norm))

Max train value after normalization: 0.9999999999885152
Min train value: 0.0
Max train value after normalization: 0.9999999999902601
Min train value: 0.0


In [57]:
print(f"Non-normalised data: {X_train_stft[:1]}")
print(f"Normalised data: {X_train_stft_norm[:1]}")

Non-normalised data: [[[[5.42520562e+01]
   [1.38138170e+02]
   [1.14104234e+02]
   ...
   [6.59093123e+01]
   [4.77410710e+01]
   [3.71320948e+00]]

  [[4.32470089e+01]
   [7.59073878e+01]
   [5.46204039e+01]
   ...
   [3.59166572e+01]
   [2.91407816e+01]
   [3.58674169e+00]]

  [[2.06129001e+01]
   [9.65369300e+00]
   [3.62894854e+00]
   ...
   [4.28695613e+00]
   [8.30676130e+00]
   [3.23280291e+00]]

  ...

  [[6.79085950e-01]
   [1.44238336e-01]
   [1.17573314e-01]
   ...
   [1.88635223e-01]
   [3.32263909e-01]
   [1.39316725e-01]]

  [[8.81099761e-01]
   [1.60037759e-01]
   [1.15180889e-01]
   ...
   [9.82244980e-02]
   [2.22017585e-01]
   [1.47221952e-01]]

  [[9.95661145e-01]
   [3.58615846e-01]
   [1.87026252e-01]
   ...
   [2.80029546e-01]
   [3.79523472e-01]
   [1.50373997e-01]]]]
Normalised data: [[[[3.75845082e-01]
   [9.57001484e-01]
   [7.90496263e-01]
   ...
   [4.56605636e-01]
   [3.30737489e-01]
   [2.57159258e-02]]

  [[2.99602980e-01]
   [5.25871517e-01]
   [3.78396

### Training

In [58]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

In [78]:
%%time

training_info_cnn_2d_model = cnn_2d_model.fit(X_train_stft_norm, y_train_stft, validation_split=0.2, epochs=30, batch_size=128, callbacks=[early_stop])

Epoch 1/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 42s 458ms/step - accuracy: 0.5426 - loss: 0.6896 - val_accuracy: 0.4241 - val_loss: 0.6985
Epoch 2/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 40s 435ms/step - accuracy: 0.5386 - loss: 0.6901 - val_accuracy: 0.4241 - val_loss: 0.6998
Epoch 3/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 44s 481ms/step - accuracy: 0.5420 - loss: 0.6892 - val_accuracy: 0.4241 - val_loss: 0.7008
Epoch 4/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 40s 440ms/step - accuracy: 0.5468 - loss: 0.6883 - val_accuracy: 0.4241 - val_loss: 0.6988
Epoch 5/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 39s 432ms/step - accuracy: 0.5351 - loss: 0.6907 - val_accuracy: 0.4241 - val_loss: 0.6955
Epoch 6/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 42s 459ms/step - accuracy: 0.5408 - loss: 0.6904 - val_accuracy: 0.4241 - val_loss: 0.7176
Epoch 7/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 40s 443ms/step - accuracy: 0.5373 - loss: 0.6905 - val_accuracy: 0.4241 - val_loss: 0.7042
Epoch 8/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 39s 426ms/step - accuracy: 0.5465 - loss: 0.6866 - val_accu

In [79]:
def plot_training_history(training_info):
  fig, axs = plt.subplots(1, 2, figsize=(16, 5))
  axs[0].plot(training_info.history['loss'], label="training set")
  axs[0].plot(training_info.history['val_loss'], label="validation set")
  axs[0].set_xlabel("Epoch")
  axs[0].set_ylabel("Loss")
  axs[0].grid(True)
  axs[0].legend()
  try:
    axs[1].plot(training_info.history['accuracy'], label="training set")
    axs[1].plot(training_info.history['val_accuracy'], label="validation set")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].grid(True)
    axs[1].legend()
  except:
    pass
  plt.show()

plot_training_history(training_info_cnn_2d_model)

In [80]:
cnn_2d_model.evaluate(X_test_stft_norm, y_test_stft)

358/358 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.5164 - loss: 0.6921


[0.6872320175170898, 0.5436188578605652]

In [81]:
y_pred = cnn_2d_model.predict(X_test_stft_norm)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_stft, y_pred_labels))
print(classification_report(y_test_stft, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step
[[3215 1893]
 [3328 3004]]
              precision    recall  f1-score   support

           0       0.49      0.63      0.55      5108
           1       0.61      0.47      0.54      6332

    accuracy                           0.54     11440
   macro avg       0.55      0.55      0.54     11440
weighted avg       0.56      0.54      0.54     11440



In [46]:
print("Predicted label distribution:", np.unique(y_pred_labels, return_counts=True))

Predicted label distribution: (array([0]), array([11440]))


In [47]:
print("True label distribution:", np.unique(y_test_stft, return_counts=True))

True label distribution: (array([0, 1]), array([5108, 6332]))


In [82]:
y_pred_probs = cnn_2d_model.predict(X_test_stft_norm)
print("Min prob:", y_pred_probs.min())
print("Max prob:", y_pred_probs.max())

358/358 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step
Min prob: 0.45536286
Max prob: 0.59533376


In [69]:
unique, counts = np.unique(y_test_stft, return_counts=True)
print(dict(zip(unique, counts)))

{np.int64(0): np.int64(5108), np.int64(1): np.int64(6332)}


### Simplified CNN model

In [91]:
def build_simplified_cnn(input_shape=(65, 25, 1)):
    input_layer = tf.keras.Input(shape=input_shape)

    x = tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu')(input_layer)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [92]:
%%time

cnn_2d_model_simplified = build_simplified_cnn(input_shape=(65, 25, 1))
training_info_cnn_2d_model_simplified = cnn_2d_model_simplified.fit(X_train_stft_norm, y_train_stft,
                                 validation_split=0.2,
                                 epochs=30,
                                 batch_size=128,
                                 callbacks=[early_stop])

Epoch 1/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.5249 - loss: 0.9771 - val_accuracy: 0.4241 - val_loss: 0.6938
Epoch 2/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.5402 - loss: 0.6730 - val_accuracy: 0.5759 - val_loss: 0.6931
Epoch 3/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.6023 - loss: 0.6503 - val_accuracy: 0.4241 - val_loss: 0.6944
Epoch 4/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - accuracy: 0.6520 - loss: 0.6211 - val_accuracy: 0.4241 - val_loss: 0.7183
Epoch 5/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.6819 - loss: 0.5982 - val_accuracy: 0.4241 - val_loss: 0.7410
Epoch 6/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - accuracy: 0.7185 - loss: 0.5713 - val_accuracy: 0.4241 - val_loss: 0.6787
Epoch 7/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - accuracy: 0.7080 - loss: 0.5731 - val_accuracy: 0.6588 - val_loss: 0.6550
Epoch 8/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 75ms/step - accuracy: 0.7140 - loss: 0.5597 - val_accuracy: 0.8086 - 

In [89]:
def plot_training_history(training_info):
  fig, axs = plt.subplots(1, 2, figsize=(16, 5))
  axs[0].plot(training_info.history['loss'], label="training set")
  axs[0].plot(training_info.history['val_loss'], label="validation set")
  axs[0].set_xlabel("Epoch")
  axs[0].set_ylabel("Loss")
  axs[0].grid(True)
  axs[0].legend()
  try:
    axs[1].plot(training_info.history['accuracy'], label="training set")
    axs[1].plot(training_info.history['val_accuracy'], label="validation set")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].grid(True)
    axs[1].legend()
  except:
    pass
  plt.show()

plot_training_history(training_info_cnn_2d_model_simplified)

In [93]:
y_pred = cnn_2d_model_simplified.predict(X_test_stft_norm)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_stft, y_pred_labels))
print(classification_report(y_test_stft, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step
[[4035 1073]
 [1251 5081]]
              precision    recall  f1-score   support

           0       0.76      0.79      0.78      5108
           1       0.83      0.80      0.81      6332

    accuracy                           0.80     11440
   macro avg       0.79      0.80      0.80     11440
weighted avg       0.80      0.80      0.80     11440



In [86]:
y_pred_probs = cnn_2d_model_simplified.predict(X_test_stft_norm)
print("Min prob:", y_pred_probs.min())
print("Max prob:", y_pred_probs.max())

358/358 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Min prob: 0.09505223
Max prob: 1.0
